In [31]:
# import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()

import requests
import re
from bs4 import BeautifulSoup, element
import time
import pandas as pd
import random
import csv


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from ecig_parsing import *


In [7]:
BASE = 'https://csvape.com' # https://csvape.com/collections/7-daze-salt-nicotine

links = ['/collections/10000-puffs-disposables', '/collections/15000-puff-disposable-vape', 
         '/collections/3000-puffs-disposables', '/collections/5000-puffs-disposables', '/collections/7-daze',
         '/collections/7-daze-salt-nicotine', '/collections/vape-accessories', 
         '/collections/accessories-brands', '/collections/adam-bomb', '/collections/adope-life', 
         '/collections/aesop-crft-eliquids', '/collections/ejuice-flavor-profile', 
         '/collections/all-in-one-starter-kits', '/collections/alternativ-salts-eliquids', 
         '/collections/american-pie-ejuice', '/collections/anml-ejuice', 
         '/collections/anml-salt-nicotine-ejuice', '/collections/vape-apparel', '/collections/aqua', 
         '/collections/aqua-nicotine-salts-eliquids-1', '/collections/aqua-nicotine-salts-eliquids', 
         '/collections/aspire-vape-starter-kits', '/collections/aspire-tanks', '/collections/atlantis', 
         '/collections/bandb-la', '/collections/bad-drip-ejuice', '/collections/baked-vapes-eliquids', 
         '/collections/bakers-man-ejuices', '/collections/barista-brew-co', '/collections/mod-batteries', 
         '/collections/vape-battery-chargers', '/collections/big-cheap-bottle-bcb-eliquids', 
         '/collections/bear-graham-eliquid', '/collections/beard-salts', '/collections/beard-vape-co-ejuice', 
         '/collections/beard-vape-co-pod-systems', '/collections/black-box-e-liquids', 
         '/collections/black-friday-40-off', '/collections/black-friday-50-off', 
         '/collections/black-friday-60-off', '/collections/black-friday-70-off', 
         '/collections/black-friday-sale', '/collections/blvk-unicorn', '/collections/boba-e-liquid', 
         '/collections/boba-vapors', '/collections/boho-vape', '/collections/bolt', 
         '/collections/bomb-bombz-eliquids', '/collections/bomb-bombz-eliquids-salt-nicotine', 
         '/collections/bomb-salts', '/collections/bomb-sauce-ejuice', '/collections/bongs', 
         '/collections/bora-e-liquids', '/collections/box-mods', '/collections/brands', 
         '/collections/breakfast-flavors', '/collections/bud-vape', '/collections/bundles-vape-starter-kits', 
         '/collections/cafe-flavors-e-liquid', '/collections/cali-pods', 
         '/collections/california-grown-e-liquids', '/collections/candy-co-ejuice', 
         '/collections/candy-flavors-e-liquid', '/collections/capsules', 
         '/collections/cecet-california-electronic-cigarette-excise-tax', 
         '/collections/charlie-s-chalk-dust', '/collections/chews', '/collections/choice', 
         '/collections/circus-cookie-ejuice', '/collections/clancys-vapors-ejuice', '/collections/clearance', 
         '/collections/cloud-nurdz', '/collections/cloud-nurdz-salts', '/collections/coastal-clouds', 
         '/collections/coastal-clouds-salts', '/collections/coil-build-tools', '/collections/coil-spill', 
         '/collections/cola-man', '/collections/cookies', '/collections/cosmic-fog-ejuice', 
         '/collections/cotton-wicks', '/collections/cream-collection', '/collections/cream-collection-salts', 
         '/collections/creams', '/collections/creamy-szns', '/collections/cuttwood-ejuice', 
         '/collections/cyber-monday', '/collections/dab-rigs', '/collections/dabbing-accessories', 
         '/collections/deja-brew', '/collections/delta-8', '/collections/dessert-flavors-e-liquid', 
         '/collections/dinner-lady', '/collections/dirty-dozen-eliquids', '/collections/disposable-pod-system', 
         '/collections/disposables', '/collections/do-not-discount', '/collections/dollar-vgod', 
         '/collections/dr-shugar-chitz', '/collections/drink-flavors-e-liquid', '/collections/drip-bar',
         '/collections/drip-cult', '/collections/drip-stix', '/collections/drplt', '/collections/dunk-d', 
         '/collections/dynamite', '/collections/e-liq-cube-ejuice', '/collections/e-liq-cube', 
         '/collections/eau-de-vape', '/collections/ecigara', '/collections/efest', 
         '/collections/ejuice-disposables', '/collections/ejuice-brands', '/collections/ejuice-bundle', 
         '/collections/ejuice-more-brands', '/collections/ejuice', '/collections/eliquid-salt-nicotine', '/collections/eloquent-ejuice', '/collections/emoji-e-liquid', '/collections/entourage-vapor', '/collections/epic-juice-ejuice', '/collections/epitome-eliquids', '/collections/exotic-bar', '/collections/ezo-eliquids', '/collections/faded-vapes-ejuice', '/collections/five-star-juice', '/collections/flavor-drips', '/collections/flavor-profile', '/collections/fogg', '/collections/four-seasons-salts', '/collections/freemax-vape-starter-kits', '/collections/freemax-tanks', '/collections/fresh-farms', '/collections/fresh-pressed', '/collections/fresh-pressed-salt-nicotine', '/collections/frozen-vape-co', '/collections/fruitly-e-juice', '/collections/fruity-flavors-e-liquid', 
         '/collections/fu-k-jerry-s-juice', '/collections/geek-vape-accessories', '/collections/geek-vape-vape-starter-kits', '/collections/geekvape', '/collections/glas-vapor-ejuice', '/collections/glas-vapor-salt-nic', '/collections/gonzo-vapors', '/collections/grandmas-vapery', '/collections/grinder', '/collections/gummies', '/collections/gummy-o-s', '/collections/halcyon', '/collections/halo', '/collections/hand-pipes', '/collections/hellvape', '/collections/high-voltage', '/collections/holiday-mega-sale', '/collections/holiday-sale', '/collections/horizontech', '/collections/hotcakes-ejuice', '/collections/hyppe', '/collections/iceflo', '/collections/innokin', '/collections/it-s-pixy', '/collections/it-s-pixy-salts', '/collections/jam-monster-eliquids', '/collections/jam-monster-salt-nic', '/collections/jet-fuel-ejuice', 
         '/collections/jones', '/collections/juice-dimension', '/collections/juice-head', '/collections/juice-head-salts', '/collections/just-fog', '/collections/key-juice', '/collections/kilo-ejuice', '/collections/kilo-salt-series', '/collections/krsp-ejuice', '/collections/kung-fu-vapor', '/collections/kvrma-vape-co', '/collections/legend-ejuice', '/collections/levels-e-liquid', '/collections/liquid-efx-vape', '/collections/liquids', '/collections/lite-vape', '/collections/loaded-ejuice', '/collections/lost-vape-accessories', '/collections/lost-vape-pod-systems', '/collections/low-resistance-wires', '/collections/lucid', '/collections/mad-hatter-juice', '/collections/makkover', '/collections/marmalade', '/collections/mastermind', '/collections/mechanical-mods', 
         '/collections/mechanical-tube-mods', '/collections/menthol-flavors-e-liquid', '/collections/met4', '/collections/mi-pod', '/collections/mk-vape-eliquids', '/collections/mnky-bizzness', '/collections/vape-mods', '/collections/moku-ejuice', '/collections/moo-eliquid', '/collections/myle', '/collections/myle-pods', '/collections/naked-100', '/collections/naked-100-salt-nicotine', '/collections/native-wicks', '/collections/newarrivals', '/collections/new-releases', '/collections/nicotine-salt-ejuice-brands', '/collections/nitecore', '/collections/njoy-artist-collection', '/collections/nktr-sour', '/collections/offset-vapors', '/collections/og-ejuice', '/collections/one-up', '/collections/open-box', '/collections/opulence-vapor', '/collections/orgnx-ejuice-brands', '/collections/orgnx-salt-nicotine-brands', '/collections/pacha-mama-ejuice-brands', '/collections/pacha-mama-salt-nicotine-brands', '/collections/pancake-house', '/collections/pioneer4you', '/collections/plus-one-vapors', '/collections/pnd', '/collections/pod-devices', '/collections/pod-ejuices', '/collections/pod-systems', '/collections/pop-bars-eliquid', '/collections/popular-vape-categories', '/collections/popvapor', '/collections/pre-filled-pod-systems', '/collections/premade-rebuildable-coils', '/collections/premium', '/collections/propaganda-ejuice', '/collections/puff-bar', '/collections/rda', '/collections/rdta', '/collections/reds-ejuice', '/collections/refresh-premium-ejuice', '/collections/refreshing-milk-bar-liquids', '/collections/replacement-atomizer-coils-tanks', '/collections/replacement-atomizer-coils-vape-accessories', '/collections/replacement-glass', '/collections/replacement-pods-pod-systems', '/collections/replacement-pods-vape-accessories', '/collections/replacement-vaporizer', '/collections/replacement-vaporizers', '/collections/ripe-collection', '/collections/rocka-mystic', '/collections/rogue-e-liquid',
         '/collections/rolling-papers-and-supplies', '/collections/rooks', '/collections/route-66', '/collections/rta', '/collections/salt-factory-e-liquids', '/collections/salt-nicotine-brands', '/collections/saltnic', '/collections/saltnic-labs', '/collections/salty-fog-e-liquids', '/collections/salty-man-vapor', '/collections/salty-man-vapor-ejuice', '/collections/salty-man-vapor-salt-nic', '/collections/saucy', '/collections/sengoku-vapor', '/collections/sengoku-vapor-salt-nicotine', '/collections/shijin', '/collections/shijin-vapor-salt-nic', '/collections/sideshow-serums-ejuice', '/collections/sigelei', '/collections/signature-premium-eliquids', '/collections/silverback', '/collections/sinbad-vapors', '/collections/smok-pod-systems', '/collections/smok-tanks', '/collections/smok-vape', '/collections/smok-vape-accessories', '/collections/smok-vape-starter-kits', '/collections/smoke', '/collections/smoke-shop', '/collections/snow-wolf', '/collections/snow-wolf-ejuice', '/collections/solace-pod-eliquid', '/collections/sour-flavors', '/collections/space-jam-ejuice', '/collections/spice-flavors', '/collections/vape-starter-kits', '/collections/steep-pop-deez', '/collections/stig-ejuice', '/collections/stig-pod-systems', '/collections/sub-ohm-tanks', '/collections/sugoi-vapor', '/collections/suorin', '/collections/svrf', '/collections/taboo-e-liquid', '/collections/taffy-king-e-liquid', '/collections/tank-replacement-coils', '/collections/tanks', '/collections/tanks-brands', '/collections/tasty-o-s', '/collections/th-t-s-lt', '/collections/the-bubble-co', '/collections/the-drip-co', '/collections/the-finest', '/collections/the-finest-salt-nicotine', '/collections/the-mamasan', '/collections/the-mamasan-salt-nicotine', '/collections/the-milkman-ejuice', '/collections/the-original-taco-mang', '/collections/the-originals', '/collections/the-pier', '/collections/the-slushy-man', '/collections/the-standard-vape', '/collections/the-standard-vape-co', '/collections/the-vaping-monkey', '/collections/thr33-monkeyz', '/collections/tobacco-flavors-e-liquid', '/collections/topdevices', '/collections/topeliquids', '/collections/triumphant-premium-juice-ejuice', '/collections/twonk-juice', '/collections/uncle-junks', '/collections/usa-vape-lab', '/collections/uwell', '/collections/vape-belly', '/collections/bundles', '/collections/vape-carrying-cases', '/collections/vape-holder', '/collections/vapekitbuilder', '/collections/vapeccino', '/collections/vapetasia', '/collections/vapetasia-salts', '/collections/vaping-lust', '/collections/vaporesso-pod-systems', '/collections/vaporesso-vape-accessories', 
         '/collections/vaporesso-vape-starter-kits', '/collections/vaporizers', '/collections/vaporizers-1', '/collections/variable-voltage-mods', '/collections/vgod', '/collections/vgod-devices', '/collections/vgod-ejuice', '/collections/vgod-saltnic', '/collections/vgod-tricklyfe', '/collections/voopoo-pod-systems', '/collections/voopoo-tanks', '/collections/voopoo-vape-accessories', '/collections/voopoo-vape-starter-kits', '/collections/wffl-e-liquid', '/collections/whip-dream', '/collections/whip-it-eliquids', '/collections/xtra',
         '/collections/yami-vapor', '/collections/yami-vapor-salt-nicotine', '/collections/yogi', '/collections/yogi-salt-nicotine', '/collections/zabba-eliquid', '/collections/zooka-vape']

print(len(links))

358


In [194]:
found = dict()

In [223]:
def get_html(url, clicked=False, closed=False, elements=True):

    try:
        driver = webdriver.Firefox()
        driver.get(url)
        last_n = 0
        hrefs = set()
        button_id = 'ac-ag-yes-button'

        html = driver.page_source
        time.sleep(1)
        same_count = 0
        if not clicked:
            wait = WebDriverWait(driver, 25)  # 10 seconds timeout
            button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))  # Use ID, XPATH, or other locator

            # Click the button
            button.click()
            clicked = True
        if not closed:
            wait = WebDriverWait(driver, 25)  # 10 seconds timeout
            button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'close')))  # Use ID, XPATH, or other locator

            # Click the button
            button.click()
            closed = True
    

        # Set a delay for dynamic content loading
        SCROLL_PAUSE_TIME = 3  # Shorter pause for smoother scroll

        # Scroll incrementally by a small step (e.g., 500 pixels)
        scroll_increment = 750

        # Get the initial scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        first_height = last_height
        # print(last_height)

        while True:
            # Scroll down by the increment
            driver.execute_script(f"window.scrollBy(0, {scroll_increment});")

            # Wait for the page to load new content
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate the new scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # Check if new elements have appeared on the screen
            # You can check specific elements like this:
            # elements = driver.find_elements(By.CLASS_NAME, "your-element-class")

            # If the scroll height has not changed, break the loop
            #print(same_count, new_height, last_height)
            if new_height == last_height :
                same_count += 1
                if same_count >= 5:
                    break
            else:
                same_count = 0

            # Update the last height
            last_height = new_height
        html = driver.page_source

    except Exception as ex:
        print(ex)
    finally:
        if driver:
            driver.close()
        html = ''
    time.sleep(3)


    return html

def extract_options(soup, header_name):
    options = []
    header = soup.find('span', text=header_name)
    if header:
        # Find the next div with the class 'block-swatch-list' containing the options
        option_div = header.find_next('div', class_='block-swatch-list')
        if option_div:
            inputs = option_div.find_all('input', class_='block-swatch__radio')
            for option in inputs:
                options.append(option['value'])
    return options


In [238]:
from collections import Counter

product_list = list()
all_headers = list()
header_counter = Counter(all_headers)

for l in links:
    page = 1
    site_section = l.replace('/collections/', '')

    url = f'{BASE}{l}'

    if url in found:
        reqtxt = found[url]
    else:
        reqtxt = get_html(url)
        found[url] = reqtxt
    #reqtxt = get_html(url)
    soup = BeautifulSoup(reqtxt)
    products = soup.find_all('div', {'class': 'product-item'})
    print(url, len(products))

    for p in products:
        if not isinstance(p, element.Tag):
            continue
        title = p.find("a", class_="product-item__title").text.strip()
        link = p.find("a", class_="product-item__title")['href']
        spe = p.find("span", class_="price--highlight")
        rpe = p.find("span", class_="price--compare")

        if spe:
            sale_price = spe.text.strip().replace('Sale price', '')
        else:
            sale_price = None
        if rpe:
            
            reg_price = rpe.text.strip().replace('Regular price', '')
        else:
            reg_price = None
        
        
        tag = link.split('/')[-1]
        full_link = f'{BASE}{link}'
        
        img_urls = []

        if full_link in found:
            reqtxt = found[full_link]
        else:
            # print(full_link)
            reqtxt = get_html(full_link, elements=False)
            found[full_link] = reqtxt
        psoup = BeautifulSoup(reqtxt)

        img_tag = psoup.find('img', class_='product-gallery__image')
        
        if img_tag and 'data-srcset' in img_tag.attrs:
            spl =  img_tag['data-srcset'].split(',')
            for s in spl:
                if s.strip().startswith('http'):
                    img_urls.append(s.strip())
                elif s.strip().startswith('//'):
                    img_urls.append('http:' + s.strip())

        elif img_tag and 'data-zoom' in img_tag.attrs:
            # Extract the URL from the data-zoom attribute
            spl =  img_tag['data-zoom'].split(',')
            for s in spl:
                if s.strip().startswith('http'):
                    img_urls.append(s.strip())
                elif s.strip().startswith('//'):
                    img_urls.append('http:' + s.strip())

            
        else:
            print('NO IMAGE URL', full_link)

        container = psoup.find('div', {'class': 'container container--flush'})
        if container:
            items_to_remove = container.find('path')
            if items_to_remove:
                items_to_remove.decompose()
            remove = container.find('script')
            if remove:
                remove.decompose()

            remove = container.find('nav')
            if remove:
                remove.decompose()

            remove = container.find('noscript')
            if remove:
                remove.decompose()
            items_to_remove = container.find('div', {'class': 'product-block-list__item--reviews'})
            if items_to_remove:
                items_to_remove.decompose()

            remove = container.find('li', {'class': 'social-media__item social-media__item--pinterest'})
            if remove:
                remove.decompose()
            remove = container.find('li', {'class': 'social-media__item social-media__item--twitter'})
            if remove:
                remove.decompose()

            remove = container.find('li', {'class': 'social-media__item social-media__item--facebook'})
            if remove:
                remove.decompose()
            items_remove = container.find('div', {'class': 'product-form__payment-container'})
            if items_remove:
                items_remove.decompose()

            remove = container.find('div', {'class': "product-block-list__item product-block-list__item--shipping"})
            if remove:
                remove.decompose()

            title = container.find('h1', class_='product-meta__title').text.strip()

#             # Extract price
#             price = container.find('span', class_='price price--highlight').text.strip()
#             compare_price = container.find('span', class_='price price--compare').text.strip()

            stock_element = container.find('span', class_='product-form__inventory')

            # Check the stock status text
            if stock_element:
                stock_status = stock_element.text.strip()
            else:
                stock_status = ""

            # Extract product description
            desc_section = container.find(lambda tag: tag.name in ['strong','h1', 'h2', 'h3', 'h4', 'h5'] and tag.text.startswith('Description'))
            description = ''
            warning = ''
            package_contents = ''
            ingredients = ''
            key_features = ''
            flavors = ''
            specifications = ''
            performance = ''
            disposable = ''
            why = ''
            preference = ''
            innovation = ''
            enjoyment = ''
            compatibility = ''
            coil = ''
            battery = ''
            puffs = ''
            nicotine = ''
            e_liquid = ''
            prefilled = ''
            devices = ''
            airflow = ''
            heating_element = ''
            operation = ''
            charging = ''
            screen = ''

            section_strings = {
                'Display Screen': 'screen',
                'Charging': 'charging',
                'Operation': 'operation',
                'Prefilled Capacity': 'prefilled',
                'Compatibility': 'compatibility',
                'Ingredients': 'ingredients',
                'INCLUDES': 'package_contents',
                "What's Included": 'package_contents',
                'Package Content': 'package_contents',
                'Package Contents': 'package_contents',
                'Package Includes': 'package_contents',
                'Included in package': 'package_contents',
                'Included in the Package': 'package_contents',
                'Included': 'package_contents',
                'Includes': 'package_contents',
                "What’s Included": 'package_contents',
                'CALIFORNIA PROPOSITION 65 - Warning': 'warning',
                'Warning': 'warning',
                'AVAILABLE OPTIONS': 'key_features',
                'Key Features': 'key_features',
                'Features': 'key_features',
                'Flavors': 'flavors',
                'Flavor Selection': 'flavors',
                'Flavor Options': 'flavors',
                'Cutting-Edge Design and Technology:': 'key_features',
                'Superior Versatility and Performance:': 'key_features',
                'Flavor Selection:': 'flavors',
                'WARNING': 'warning',
                'Description': 'description',
                'Product Specifications': 'specifications',
                'Available Flavors': 'flavors',
                'Flavor Profiles': 'flavors',
                'Exceptional Flexibility and Performance': 'performance',
                'Disposable': 'disposable',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Tailored to Your Preference': 'preference',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Enjoy My Shisha on the Go': 'enjoyment',
                'The Ultimate Vaping Experience': 'enjoyment',
                'Elevate Your Vaping Experience': 'enjoyment',
                'Why Choose the Juice Head 30K Disposable?': 'why',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Hotbox Features': 'key_features',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Experience the Ultimate Vaping Innovation': 'innovation',
                'Flavor Details': 'flavors',
                'Advanced Dual Mesh Coil': 'coil',
                'Coil Specifications': 'coil',
                'Puff Count': 'puffs',
                'Max Puffs': 'puffs',
                'Nicotine Strength': 'nicotine',
                'Battery Capacity': 'battery',
                'E-liquid Capacity': 'e_liquid',
                'Maximum Puffs': 'puffs',
                'Available Flavor Profiles': 'flavors',
                'Prefilled': 'prefilled',
                'Compatible Devices': 'devices',
                'Devices': 'devices',
                'California Proposition 65 Warning': 'warning',
                'Melli Flavors': 'flavors',
                'Airflow': 'airflow',
                'Heating Element': 'heating_element'
                
            }
            
            section_map = {
                'description': description,
                'warning': warning,
                'package_contents': package_contents,
                'ingredients': ingredients,
                'key_features': key_features,
                'flavors': flavors,
                'specifications': specifications,
                'performance': performance,
                'disposable': disposable,
                'why': why,
                'innovation': innovation,
                'preference': preference,
                'enjoyment': enjoyment,
                'compatibility': compatibility,
                'coil': coil,
                'battery': battery,
                'puffs': puffs,
                'nicotine': nicotine,
                'e_liquid': e_liquid,
                'prefilled': prefilled,
                'devices': devices,
                'airflow': airflow,
                'heating_element': heating_element,
                'operation': operation,
                'charging': charging,
                'screen': screen
            }

            cur_section = 'description'
            if desc_section:
                siblings = desc_section.parent.find_next_siblings()
                for s in siblings:
                    headers = [tag.text for tag in s.find_all(lambda tag: tag.name in ['strong','h1', 'h2', 'h3', 'h4', 'h5'])]
                    stext = s.get_text(separator='\n').split('\n')
                    for s in stext:
                        s = s.strip()
                        if s == '':
                            continue
                        s_no_colon = s.replace(':', '').strip()
                        if s in section_strings:
                            cur_section = section_strings[s]
                        elif s_no_colon in section_strings:
                            cur_section = section_strings[s_no_colon]
                        elif s.lower().startswith('why') and s.endswith('?'):
                            cur_section = 'why'
                        elif 'features:' in s.lower():
                            cur_section = 'key_features'
                        elif s in headers:
                            all_headers.append(s)
                            header_counter = Counter(all_headers)
                            if header_counter[s] > 5:
                                print(s, full_link)
                            cur_section = 'description'
                            section_map[cur_section] += f'{s}\n\n'
                        else:
                            section_map[cur_section] += f'{s}\n'



            flavor_blocks = container.find_all('div', class_='block-swatch')

            flavors = []

            # Loop through each block and extract flavor name and disabled status
            for block in flavor_blocks:
                flavor_name = block.find('span', class_='block-swatch__item-text').text
                is_disabled = 'block-swatch--disabled' in block.get('class', [])
                flavors.append(flavor_name)
                
        nicotine_strengths = extract_options(psoup, "Nicotine Strength")
        bottle_sizes = extract_options(psoup, "Bottle Size")
        
        n = 0
        for i in img_urls:
            n += 1
            download_image(i, tag, save_dir='data-2024-12/cvsvape_images')
            # these seem to be the same
            break



        # Extracting product information
        product_data = {
            'tag': tag,
            "title": title,
            "link": full_link,
            "sale_price": sale_price,
            "regular_price": reg_price,
            "image_urls": ','.join(img_urls),
            'compare_price': compare_price,
            'flavor_list': flavors,
            'nicotine_strengths': nicotine_strengths,
            'bottle_sizes': bottle_sizes,
            "stock_status": stock_status,
            'site_category': site_section
        }
        desc_fields = ''
        for s, v in section_map.items():
            if 'description' not in s:
                s = s + '_description'
            product_data[s] = v.replace('\xa0', ' ').strip()
            if product_data[s] != '':
                desc_fields += f'\n{product_data[s]}'
            
        feats = list()
        #print(desc_fields)
        feat = find_features(desc_fields)
        any_found, puffs_res, nico_res, ml_res, flav_text, dev_text = feat

        if any_found:
            #print(feat)
            feats.append(feat)
        disposable,recharge,battery,mesh,usb,adjustable,found_flavs = features_to_cats(feats)
        product_data['disposable'] = disposable
        product_data['rechargeable'] = recharge
        product_data['battery'] = battery
        product_data['mesh'] = mesh
        product_data['usb'] = usb
        product_data['adjustable'] = adjustable

        #print(product_data)
        
        product_list.append(product_data)




random.choice(product_list)

https://csvape.com/collections/10000-puffs-disposables 17
File already exists at cvsvape_images/geek-bar-pulse-15000-puff-disposable.png. Skipping download.
File already exists at cvsvape_images/geek-bar-pulse-x-25k-puff-disposable-vape.jpg. Skipping download.
File already exists at cvsvape_images/smok-spaceman-prism-20000-puffs-disposable-vape.jpg. Skipping download.
File already exists at cvsvape_images/my-shisha-10000-puffs-disposable-vape-freebase.png. Skipping download.
File already exists at cvsvape_images/flum-mello-20000-puff-disposable-vape.jpg. Skipping download.
File already exists at cvsvape_images/geek-bar-skyview-25000-puff-disposable-vape.png. Skipping download.
File already exists at cvsvape_images/lost-mary-mt15000-disposable-vape.jpg. Skipping download.
File already exists at cvsvape_images/the-puff-brands-hotbox-luxe-12000-puff-disposable.webp. Skipping download.
File already exists at cvsvape_images/craftbox-v-play-20k-puffs-disposable-vape.png. Skipping download.
F

File already exists at cvsvape_images/pod-juice-pod-5500-5-5-tf-disposable-device.jpg. Skipping download.
File already exists at cvsvape_images/kuz-9000-puff-disposable-device.png. Skipping download.
https://csvape.com/collections/7-daze 32
File already exists at cvsvape_images/daze-fusion-series-raspberry-green-apple-iced.png. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-raspberry-green-apple.png. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-kiwi-passionfruit-guava-iced.png. Skipping download.
File already exists at cvsvape_images/reds-e-juice-watermelon-iced-100ml.png. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-pineapple-mango-orange-iced.jpg. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-strawberry-banana-apple-iced.png. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-kiwi-passionfruit-guava.jpg. Skipping download.
File already exis

Image saved: cvsvape_images/smok-v12-t8-tfv12-replacement-coil-3-pack.jpg
Image saved: cvsvape_images/vaporesso-gtr-mesh-3pack.png
Image saved: cvsvape_images/uwell-crown-v3-replacement-coil-4-pack.jpg
Image saved: cvsvape_images/burnbud-all-in-one-accessory.jpg
Image saved: cvsvape_images/grenco-science-dr-greenthumb-x-g-pen-connect-vaporizer.png
Image saved: cvsvape_images/bee-line-organic-hemp-wick-original.jpg
Image saved: cvsvape_images/vaporesso-gtx-go-replacement-26-pod-2-pack.jpg
Image saved: cvsvape_images/raw-mini-rolling-tray.jpg
Image saved: cvsvape_images/ooze-glass-globe.jpg
https://csvape.com/collections/accessories-brands 5
File already exists at cvsvape_images/smok-v8-baby-t6-replacement-coils.jpg. Skipping download.
File already exists at cvsvape_images/smok-v8-baby-m2-replacement-coil-5-pack.jpg. Skipping download.
File already exists at cvsvape_images/smok-v8-baby-t8-beast-coil-5-pack.jpg. Skipping download.
File already exists at cvsvape_images/smok-v12-t8-tfv12-re

Image saved: cvsvape_images/yogi-vanilla-tobacco-granola-bar.png
Image saved: cvsvape_images/yogi-salt-original-granola-bar.png
Image saved: cvsvape_images/yogi-60ml-0mg.jpg
Image saved: cvsvape_images/jam-monster-salts-mixed-berry.jpg
Salt nicotine ejuices are for pod systems ONLY and should not be used in sub-ohm applications. https://csvape.com/products/the-salty-one-strawberry
File already exists at cvsvape_images/the-salty-one-strawberry.jpg. Skipping download.
https://csvape.com/collections/bud-vape 0
https://csvape.com/collections/bundles-vape-starter-kits 0
https://csvape.com/collections/cafe-flavors-e-liquid 9
Image saved: cvsvape_images/the-finest-eliquid-signature-edition-tiramisu-custard.jpg
Image saved: cvsvape_images/the-finest-e-liquid-salt-nic-edition-tiramisu-custard.jpg
File already exists at cvsvape_images/the-finest-e-liquid-salt-nic-edition-gold-reserve.jpg. Skipping download.
File already exists at cvsvape_images/glas-basix-strawberry-milk.jpg. Skipping download.


Image saved: cvsvape_images/cloud-nurdz-tfn-salts-blue-raspberry-lemon.jpg
Image saved: cvsvape_images/glas-basix-blue-razz.jpg
File already exists at cvsvape_images/beard-120ml-0mg.jpg. Skipping download.
Image saved: cvsvape_images/sweet-collection-purple-sweet.jpg
https://csvape.com/collections/capsules 0
https://csvape.com/collections/cecet-california-electronic-cigarette-excise-tax 112
Image saved: cvsvape_images/daze-7obacco.jpg
Image saved: cvsvape_images/7-daze-egge-disposable.jpg
File already exists at cvsvape_images/daze-fusion-series-banana-cantaloupe-honeydew.png. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-banana-cantaloupe-honeydew-iced.png. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-grape-apple-aloe.png. Skipping download.
File already exists at cvsvape_images/daze-fusion-series-grape-apple-aloe-iced.png. Skipping download.
Image saved: cvsvape_images/daze-fusion-series-grapefruit-orange-mango.png
File alre

Image saved: cvsvape_images/barista-brew-co-coffee-frozen-watermelon-refresher.jpg
Image saved: cvsvape_images/barista-brew-co-maple-bar-donut.jpg
Image saved: cvsvape_images/barista-brew-co-salted-caramel.jpg
Image saved: cvsvape_images/barista-brew-co-smores-mocha-breeze.jpg
Image saved: cvsvape_images/barista-brew-co-strawberry-watermelon-refresher.jpg
Image saved: cvsvape_images/barista-brew-co-white-chocolate-mocha.jpg
File already exists at cvsvape_images/beard-no-05.jpg. Skipping download.
We recommend using this ejuice for pod systems only. Salt nicotine ejuices are NOT for sub-ohm use. https://csvape.com/products/beard-salts-no-00
File already exists at cvsvape_images/beard-salts-no-00.jpg. Skipping download.
We recommend using this ejuice for pod systems only. Salt nicotine ejuices are NOT for sub-ohm use. https://csvape.com/products/beard-salts-no-05
File already exists at cvsvape_images/beard-salts-no-05.jpg. Skipping download.
We recommend using this ejuice for pod systems

Image saved: cvsvape_images/cloud-nurdz-salts-verry-berry-hibiscus-iced.png
Image saved: cvsvape_images/cloud-nurdz-salts-strawberry-kiwi.png
Image saved: cvsvape_images/cloud-nurdz-salts-guava-passionfruit.png
Image saved: cvsvape_images/cloud-nurdz-salts-cherry-apple-iced.png
We recommend using this ejuice for pod systems only. Salt nicotine ejuices are NOT for sub-ohm use. https://csvape.com/products/cloud-nurdz-salts-sour-strawberry
File already exists at cvsvape_images/cloud-nurdz-salts-sour-strawberry.jpg. Skipping download.
Image saved: cvsvape_images/cloud-nurdz-salts-peach-dragonfruit.png
Image saved: cvsvape_images/cloud-nurdz-salts-strawberry-kiwi-iced.png
Image saved: cvsvape_images/cloud-nurdz-salts-peach-dragon-fruit.png
Image saved: cvsvape_images/cloud-nurdz-salts-cherry-apple.png
We recommend using this ejuice for pod systems only. Salt nicotine ejuices are NOT for sub-ohm use. https://csvape.com/products/cloud-nurdz-salts-strawberry-mango
File already exists at cvsvap

Image saved: cvsvape_images/custard-monster-salts-vanilla.jpg
Image saved: cvsvape_images/custard-monster-blueberry.jpg
Image saved: cvsvape_images/glas-e-liquid-sugar-cookie.jpg
Salt nicotine ejuices are for pod systems ONLY and should not be used in sub-ohm applications. https://csvape.com/products/custard-monster-salts-strawberry
Image saved: cvsvape_images/custard-monster-salts-strawberry.jpg
Image saved: cvsvape_images/the-mamasan-salts-mama-melon.jpg
Image saved: cvsvape_images/the-mamasan-salts-purple-cheesecake.jpg
File already exists at cvsvape_images/coastal-clouds-maple-butter.jpg. Skipping download.
Image saved: cvsvape_images/jam-monster-apple.jpg
**Not for sub-ohm use** https://csvape.com/products/the-finest-e-liquid-salt-nic-edition-gold-reserve
File already exists at cvsvape_images/the-finest-e-liquid-salt-nic-edition-gold-reserve.jpg. Skipping download.
Image saved: cvsvape_images/jam-monster-strawberry.jpg
Image saved: cvsvape_images/vapetasia-killer-kustard-lemon.jpg

{'tag': 'jam-monster-salts-peach',
 'title': 'Jam Monster Salts Peach',
 'link': 'https://csvape.com/products/jam-monster-salts-peach',
 'sale_price': '$17.00',
 'regular_price': '$22.95',
 'image_urls': 'http://csvape.com/cdn/shop/products/jam_monster_salt_peach_400x.jpg?v=1608691260 400w,http://csvape.com/cdn/shop/products/jam_monster_salt_peach_500x.jpg?v=1608691260 500w,http://csvape.com/cdn/shop/products/jam_monster_salt_peach_600x.jpg?v=1608691260 600w,http://csvape.com/cdn/shop/products/jam_monster_salt_peach_700x.jpg?v=1608691260 700w,http://csvape.com/cdn/shop/products/jam_monster_salt_peach_800x.jpg?v=1608691260 800w,http://csvape.com/cdn/shop/products/jam_monster_salt_peach_900x.jpg?v=1608691260 900w,http://csvape.com/cdn/shop/products/jam_monster_salt_peach_1000x.jpg?v=1608691260 1000w,http://csvape.com/cdn/shop/products/jam_monster_salt_peach_1100x.jpg?v=1608691260 1100w',
 'compare_price': 'Regular price$30.00',
 'flavor_list': ['24mg (2.4%)', '48mg (4.8%)', '30ml'],
 'ni

In [225]:
#https://csvape.com/collections/5000-puffs-disposables
len(product_list)

621

In [229]:
import csv
with open('data-2024-12/csvape_scrape.csv', mode='w') as file:
    # Create a DictWriter object
    writer = csv.DictWriter(file, fieldnames=product_list[0].keys())

    # Write the header (column names)
    writer.writeheader()

    # Write the rows (each dictionary in the data list)
    writer.writerows(product_list)
